In [1]:
from azureml.core import Workspace

c:\Users\Vikas.Yetintala\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
ws= Workspace.create(name='Azureml-SDK-brillio',
                     subscription_id="a4df42c9-c1d2-44c4-b2c2-05edb4b27aad",
                     resource_group="RG-MSFT-ML-POC-01",
                     create_resource_group=True,
                     location="")
ws.write_config(path="./config")

KeyboardInterrupt: 

In [ ]:
ws=Workspace.from_config("./config")

In [ ]:
from azureml.core import Datastore
az_store = Datastore.register_azure_blob_container(workspace=ws,
                                                    datastore_name="azure_sdk_blob01",
                                                    account_name="vikaspoctitanic",
                                                    container_name="msbrilliopoc",
                                                    account_key="be/0iIZOayOkNh4ouG4Z6S4JR7fgVM9+R2bpLRcMwO3Z5mne0GEas/ZUNLDldRYqo4/oPDcatLge+AStdxl7hQ==")

In [ ]:
az_store= Datastore.get(ws,"azure_sdk_blob01")

In [ ]:
print(az_store)

{
  "name": "azure_sdk_blob01",
  "container_name": "msbrilliopoc",
  "account_name": "vikaspoctitanic",
  "protocol": "https",
  "endpoint": "core.windows.net"
}


In [ ]:
from azureml.core import Dataset
csv_path= [(az_store,"titanic.csv")]
dataset= Dataset.Tabular.from_delimited_files(path=csv_path)
#register the dataset
dataset = dataset.register(workspace=ws,
                            name="Titanic_vikas",
                            create_new_version=True)

In [ ]:
ws_list=Workspace.list(subscription_id="a4df42c9-c1d2-44c4-b2c2-05edb4b27aad")
ws_list=list(ws_list)
az_default_store= ws.get_default_datastore()
az_dataset=Dataset.get_by_name(ws,"Titanic_vikas")
ds_list=list(ws.datasets.keys())
for items in ds_list:
    print(items)

Titanic_vikas
Loandataset
Loan application using sdk
TD-AzureMl_Design-Clean_Missing_Data-Cleaning_transformation-59449c1b
MD-AzureMl_Design-Train_Model-Trained_model-d587765b
titanic_dataset
loanapplications
ms_batch_output
mlops_data_ms_batch_5
mlops_data_ms_batch_4
mlops_data_ms_batch_3
mlops_data_ms_batch_2
mlops_data_ms_batch_1
mlpoc-dataset-01


In [ ]:
ws=Workspace.from_config("./config")
az_store= Datastore.get(ws,"azure_sdk_blob01")
az_dataset=Dataset.get_by_name(ws,"Titanic_vikas")
az_default_store= ws.get_default_datastore()

In [ ]:
df= az_dataset.to_pandas_dataframe()
df_sub=df[["Pclass","Sex","Age","SibSp","Parch","Fare","Cabin","Embarked","Survived"]]

In [ ]:
df_sub.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived
0,3,male,22.0,1,0,7.2500,None,S,0
1,1,female,38.0,1,0,71.2833,C85,C,1
2,3,female,26.0,0,0,7.9250,None,S,1
3,1,female,35.0,1,0,53.1000,C123,S,1
4,3,male,35.0,0,0,8.0500,None,S,0


In [ ]:
az_ds_from_df = Dataset.Tabular.register_pandas_dataframe(dataframe=df_sub,
                                                          target=az_store,
                                                          name="Titanic_selected_columns")

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/1e6e0889-7b6b-4060-976b-2694d9b69117/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [ ]:
files_list=["./data/test.csv","./data/test1.csv"]
files_list= az_store.upload_files(files=files_list,
                                   target_path="folder_name/",
                                relative_root="./data/",
                                overwrite=True)

In [ ]:
az_store.upload(src_dir="./data",
                target_path="folder_name/data",
                overwrite=True)

# creating custom environment

In [ ]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

# Hyper parameter Tuning

In [ ]:
myenv= Environment(name="MyEnvironment")
myenv_dep= CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults','azureml-interpret'])
myenv.python.conda_dependencies = myenv_dep
myenv.register(ws)

{
    "assetId": "azureml://locations/eastus/workspaces/3e3bfef7-2a2b-41b0-9796-e1cdf285aece/environments/MyEnvironment/versions/5",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20221010.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "MyEnvironment",
    "pyt

In [ ]:
from azureml.core.compute import AmlCompute 
cluster_name= "comput1"
if cluster_name not in ws.compute_targets:
    compute_config= AmlCompute.provisioning_configuration(vm_size="",
                                                        max_nodes=2)
    cluster = AmlCompute.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion()
else:
    cluster= ws.compute_targets[cluster_name]
print(cluster," compute cluster found.. using it....")

AmlCompute(workspace=Workspace.create(name='aml-msft-ml-poc1', subscription_id='a4df42c9-c1d2-44c4-b2c2-05edb4b27aad', resource_group='RG-MSFT-ML-POC-01'), name=comput1, id=/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourceGroups/RG-MSFT-ML-POC-01/providers/Microsoft.MachineLearningServices/workspaces/aml-msft-ml-poc1/computes/comput1, type=AmlCompute, provisioning_state=Succeeded, location=eastus, tags={})  compute cluster found.. using it....


In [ ]:
from azureml.core import ScriptRunConfig 
script_config = ScriptRunConfig(source_directory="Training_python_scripts/",
                                script="hyperdrivescript.py",
                                environment=myenv,
                                compute_target=cluster)

In [ ]:
from azureml.train.hyperdrive import GridParameterSampling,choice
hyper_params= GridParameterSampling(
                                    {'--n_estimators': choice(10,20),
                                    '--min_samples_leaf': choice(1,2,)})

In [ ]:
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal
hyper_config = HyperDriveConfig(run_config=script_config,
                                hyperparameter_sampling=hyper_params,
                                policy=None,
                                primary_metric_name='accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=4,
                                max_concurrent_runs=1)
from azureml.core.experiment import Experiment
new_exp= Experiment(ws,"exp_name")
print("submitting the exxperiment")
new_run= new_exp.submit(hyper_config)
new_run.wait_for_completion(show_output=True)


submitting the exxperiment
RunId: HD_3640174e-d7a7-48e7-90a3-912005e9e284
Web View: https://ml.azure.com/runs/HD_3640174e-d7a7-48e7-90a3-912005e9e284?wsid=/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourcegroups/RG-MSFT-ML-POC-01/workspaces/aml-msft-ml-poc1&tid=3882b70d-a91e-468c-9928-820358bfbd73

Streaming azureml-logs/hyperdrive.txt

[2022-11-02T15:21:20.291408][GENERATOR][INFO]Trying to sample '1' jobs from the hyperparameter space
[2022-11-02T15:21:20.9732786Z][SCHEDULER][INFO]Scheduling job, id='HD_3640174e-d7a7-48e7-90a3-912005e9e284_0' 
[2022-11-02T15:21:20.948753][GENERATOR][INFO]Successfully sampled '1' jobs, they will soon be submitted to the execution target.
[2022-11-02T15:21:21.5914109Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_3640174e-d7a7-48e7-90a3-912005e9e284_0' 
[2022-11-02T15:22:20.340164][GENERATOR][INFO]Trying to sample '1' jobs from the hyperparameter space
[2022-11-02T15:22:20.569980][GENERATOR][INFO]Successfully sampled '1' jobs, they wil

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "{\"NonCompliant\":\"Process '/azureml-envs/azureml_6a7de177e158ae92b1d9af27857c54b2/bin/python' exited with code 2 and error message 'Execution failed. Process exited with status code 2. Error: Traceback (most recent call last):\\n  File \\\"hyperdrivescript.py\\\", line 15, in <module>\\n    args=parser.parse_args()\\n  File \\\"/azureml-envs/azureml_6a7de177e158ae92b1d9af27857c54b2/lib/python3.8/argparse.py\\\", line 1771, in parse_args\\n    self.error(msg % ' '.join(argv))\\n  File \\\"/azureml-envs/azureml_6a7de177e158ae92b1d9af27857c54b2/lib/python3.8/argparse.py\\\", line 2521, in error\\n    self.exit(2, _('%(prog)s: error: %(message)s\\\\n') % args)\\n  File \\\"/azureml-envs/azureml_6a7de177e158ae92b1d9af27857c54b2/lib/python3.8/argparse.py\\\", line 2508, in exit\\n    _sys.exit(status)\\nSystemExit: 2\\n\\n'. Please check the log fil

In [ ]:
best_run = new_run.get_best_run_by_primary_metric() 
print("best run id: ",best_run.id)
print(best_run.get_metrics())